In [1]:
import pandas as pd
from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer, Imputer
from copy import deepcopy

In [2]:
from sklearn.impute import SimpleImputer

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Перебор гиперпараметров модели

In [4]:
df = pd.read_csv('https://gist.githubusercontent.com/braingineer/5d15057ac482ee0130b6d0e6f9cc9311/raw/d4eefaecc98b342ec578cf3512184556e8856750/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
from sklearn.utils.metaestimators import _BaseComposition
class DataFrameMapper2(DataFrameMapper, _BaseComposition):
    def get_params(self, deep=True):
        return self._get_params('features', deep=deep)

In [9]:
full_mapper2 = DataFrameMapper2([
        ('Name', Pipeline([
                    ('name_vect', CountVectorizer()),
                    ('name_tfidf', TfidfTransformer()) 
                ])),
        ('Sex', LabelBinarizer()),
#         (['Age'], age_filling),
        ('Fare', None),
    ])

In [10]:
# build full pipeline
full_pipeline  = Pipeline([
    ('mapper', full_mapper2),
    ('clf', SGDClassifier(max_iter=15, warm_start=True))
])

    

In [11]:
full_pipeline.get_params(deep=True)

{'memory': None,
 'steps': [('mapper', DataFrameMapper2(default=False, df_out=False,
                    features=[('Name',
                               Pipeline(memory=None,
                                        steps=[('name_vect',
                                                CountVectorizer(analyzer='word',
                                                                binary=False,
                                                                decode_error='strict',
                                                                dtype=<class 'numpy.int64'>,
                                                                encoding='utf-8',
                                                                input='content',
                                                                lowercase=True,
                                                                max_df=1.0,
                                                                max_features=None,
                     

In [12]:
full_params = {'clf__alpha': [1e-2,1e-3,1e-4],
               'clf__loss':['modified_huber','hinge'],
               'clf__penalty':['l2','l1'],
               'mapper__Name__name_vect__analyzer': ['char', 'char_wb']
              }

In [13]:
gs_clf = GridSearchCV(full_pipeline, full_params, n_jobs=-1, cv=5)
gs_clf.fit(df, df['Survived'])

ValueError: dictionary update sequence element #0 has length 3; 2 is required